In [ ]:
# Dependencies
from citipy import citipy
from random import uniform
import json
import requests
from pprint import pprint
from config import api_key
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Retrive 1000 citites objects from citipy
cities = []

while len(cities) <1000:
    latitude = uniform(90, -90)
    longitude = uniform(180, -180)
    new_city = citipy.nearest_city(latitude, longitude)
    if new_city not in cities:
        cities.append(new_city)    

In [ ]:
# Building query and retreiving API query
city_info=[]
city_num = 0
for city in cities:
    city_num+=1
    base_url = "http://api.openweathermap.org/data/2.5/weather?q="
    query_url = (f"{base_url}{city.city_name},{city.country_code}&units=imperial&appid={api_key}")
    weather_response= requests.get(query_url)
    city_info.append(weather_response.json())
    print(f"City count: {city_num}\n Retrieving data for {city.city_name} in {city.country_code}.")

In [ ]:
# Pretty Print response
pprint(city_info)

In [ ]:
# Function to get values from a dictionary
def safe_get(d, *keys):
    value = d
    for key in keys:
        value = value.get(key,None)
        if value is None: break
    return value

# Using list comprehension to retrieve desired data by iterating through dicts in city_info and using function "safe_get".
weather_city_info = [
    {
        "City": safe_get(city,"name"),
        "Country": safe_get(city,"sys","country"), 
        "Latitude": safe_get(city, "coord","lat"),
        "Longitude": safe_get(city, "coord","lon"),
        "Temperature (F)": safe_get(city, "main","temp"),
        "Humidity %": safe_get(city,"main", "humidity"),
        "Cloudiness %": safe_get(city, "clouds","all"),
        "Wind Speed (mph)": safe_get(city, "wind","speed")
    } for city in city_info
]
pprint(weather_city_info)


In [ ]:
weather_city_info_df = pd.DataFrame(weather_city_info)
weather_city_info_df

In [ ]:
# Find count of empty dictionaries
len([entry for entry in weather_city_info if entry["City"] is None])

In [ ]:
# Drop rows without city information with dropna()
clean_weather_city_info_df= weather_city_info_df.dropna()
# Count number of cities with information
total_cities = clean_weather_city_info_df.count

In [ ]:
clean_weather_city_info_df

In [ ]:
# Write data frame data into csv
clean_weather_city_info_df.to_csv('Outputs/weather_data.csv', index= False, header= True)


In [ ]:
# Copy data frame for integrity
data_df = clean_weather_city_info_df.copy()


In [ ]:
# Function to make scatter plots:
column_names=["Temperature (F)", "Humidity %", "Cloudiness %", "Wind Speed (mph)"]
for type in column_names:
    plt.figure(figsize= (15,10))
    plt.scatter(data_df["Latitude"], data_df[type])
    plt.xlim([-90, 90])
    if type == "Temperature (F)":
        plt.ylim(round(min(data_df[type]),-1)-10,round(max(data_df[type]),-1)+10)
    elif type == "Wind Speed (mph)":
        plt.ylim(0,round(max(data_df[type]),-1)+10)
    else:
        plt.ylim([0,100])
    plt.title(f"Latitude vs. {type}          (03/26/2021)", fontsize=20)
    plt.xlabel("Latitude", fontsize=16)
    plt.ylabel(type, fontsize=16)
    plt.grid(color= "gray", linestyle="--", linewidth= 0.5)
    txt= (f"Relationship between latitude and {type} of {total_cities} cities taken on 03/26/21 from OpenWeather API.")
    plt.figtext(0.5, 0.01, txt, wrap=True, horizontalalignment='center', fontsize=14)
    plt.show()
    plt.savefig(f"Outputs/{type}_scatter.png")